# What makes a Tweet an indicator of an ongoing Disaster 

We all know about the "Boy who cried Wolf", and it's equivalent tweets such as "its shaking in San Francisco" which could mean its fun being in SF or an earthqauke. 

In any disaster situation avaliabiltiy of the first responder could make the difference between life and death. Last year when there was severe weather system in Bayarea, crowd sourced tweets did a better job in giving updates on flood zones then any other medium. 

For a disaster management system supporting First responder team its important to understand when a tweet signals an actual disaster.

# Business Understanding

From a business perspective, we are tasked with identifying patterns in the tweet that would indicate a disaster in progress and the first responders can respond quickly and efficiently in the form of 
- Tweets asking people to stay away form the disaster zone 
- Checking progress and providing additional support 
- Providing on time help 

### Data Mining Goals and KPI

- Build predictive model with 75% accuracy
- Build predictive model with 75% recall
- Build predictive model with 75% precision

# Data Understanding

The Data is from Kaggle (https://www.kaggle.com/competitions/nlp-getting-started/overview). Given its a getting started NLP project there are multiple articles and solutions provided online to solve it. 
List of sources I referred,
- https://www.kaggle.com/competitions/nlp-getting-started/code 
- https://www.kaggle.com/code/jessicaval/disaster-tweets-prediction-logreg-embeddings
- https://medium.com/analytics-vidhya/introduction-to-nlp-with-disaster-tweets-3b672a75748c


### The data is made of the following attributes 

- id: a unique identifier of every tweet
- keyword: a particular keyword from the tweet (this can be blank)
- location: the location the tweet was sent from (this can be blank)
- text: the text of the tweet
- target: present only in the train data, and denotes if the tweet is about a real disaster (1) or not (0)

In [249]:
import pandas as pd
import string

In [250]:
df_test = pd.read_csv('nlp-getting-started/test.csv')
df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, stay safe everyone."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all"
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [251]:
df = pd.read_csv('nlp-getting-started/train.csv')
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


In [252]:
df.shape

(7613, 5)

In [253]:
categorical = [var for var in df.columns if df[var].dtype in ['object', 'category', 'string']]
print('There are {} categorical variables\n'.format(len(categorical)))
print('The categorical variables are :', categorical)

There are 3 categorical variables

The categorical variables are : ['keyword', 'location', 'text']


In [254]:
df[categorical].head()

,keyword,location,text
0,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
1,NaN,NaN,Forest fire near La Ronge Sask. Canada
2,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected
3,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California"
4,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school


In [255]:
df.keyword.value_counts()

fatalities               45
deluge                   42
armageddon               42
sinking                  41
damage                   41
                         ..
forest%20fire            19
epicentre                12
threat                   11
inundation               10
radiation%20emergency     9
Name: keyword, Length: 221, dtype: int64

In [256]:
df.location.value_counts()

USA                    104
New York                71
United States           50
London                  45
Canada                  29
                      ... 
MontrÌ©al, QuÌ©bec       1
Montreal                 1
ÌÏT: 6.4682,3.18287      1
Live4Heed??              1
Lincoln                  1
Name: location, Length: 3341, dtype: int64

### Null value Count

In [257]:
df[categorical].isnull().sum()

keyword       61
location    2533
text           0
dtype: int64

### Frequency distribution of categorical variables

In [258]:
for var in categorical: 
    print(df[var].value_counts()/np.float(len(df)))

fatalities               0.005911
deluge                   0.005517
armageddon               0.005517
sinking                  0.005386
damage                   0.005386
                           ...   
forest%20fire            0.002496
epicentre                0.001576
threat                   0.001445
inundation               0.001314
radiation%20emergency    0.001182
Name: keyword, Length: 221, dtype: float64
USA                    0.013661
New York               0.009326
United States          0.006568
London                 0.005911
Canada                 0.003809
                         ...   
MontrÌ©al, QuÌ©bec     0.000131
Montreal               0.000131
ÌÏT: 6.4682,3.18287    0.000131
Live4Heed??            0.000131
Lincoln                0.000131
Name: location, Length: 3341, dtype: float64
11-Year-Old Boy Charged With Manslaughter of Toddler: Report: An 11-year-old boy has been charged with manslaughter over the fatal sh...        0.001314
He came to a land which was engul

/var/folders/3y/h66zz12x4p9_7hpzsntxf_6m0000gn/T/ipykernel_25624/2172132597.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  print(df[var].value_counts()/np.float(len(df)))
/var/folders/3y/h66zz12x4p9_7hpzsntxf_6m0000gn/T/ipykernel_25624/2172132597.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  print(df[var].value_counts()/np.floa

### Remove Duplicate tweets 

In [259]:
#Find all the duplicated Tweets
df_duplicated = df_train[df_train.duplicated(["text"], keep = False)] #179 rows
df_duplicated.shape

(179, 5)

In [260]:
df = df.drop_duplicates(subset=['keyword','location','text'])
df.shape

(7552, 5)

### Remove tweets labelled more then once 

In [261]:
#Find all the tweets that have been labeled more than once
pd.set_option('display.max_colwidth', None) #to display the whole text
df_errors = df_train.groupby('text').filter(lambda x: x['target'].nunique() > 1, display)
df_errors.head(20)

,id,keyword,location,text,target
610,881,bioterrorism,NaN,To fight bioterrorism sir.,1
624,898,bioterrorism,NaN,To fight bioterrorism sir.,0
630,907,bioterrorism,NaN,To fight bioterrorism sir.,1
634,916,bioterrorism,NaN,To fight bioterrorism sir.,0
1197,1723,buildings%20burning,"Mackay, QLD, Australia",Mmmmmm I'm burning.... I'm burning buildings I'm building.... Oooooohhhh oooh ooh...,1
1214,1752,buildings%20burning,"Epic City, BB.",I Pledge Allegiance To The P.O.P.E. And The Burning Buildings of Epic City. ??????,0
1221,1760,buildings%20burning,dallas,like for the music video I want some real action shit like burning buildings and police chases not some weak ben winston shit,1
1331,1922,burning%20buildings,"Mackay, QLD, Australia",Mmmmmm I'm burning.... I'm burning buildings I'm building.... Oooooohhhh oooh ooh...,0
1349,1950,burning%20buildings,dallas,like for the music video I want some real action shit like burning buildings and police chases not some weak ben winston shit,0
1365,1968,burning%20buildings,"Epic City, BB.",I Pledge Allegiance To The P.O.P.E. And The Burning Buildings of Epic City. ??????,1


In [262]:
df_errors.shape

(55, 5)

In [263]:
# remove the errors from the training dataset 
df = df[~df.id.isin(df_errors.id)]

In [264]:
df.shape

(7520, 5)

In [265]:
#Check that all duplicates are removed
null = df_train["target"].isnull().sum()
print(null)

0


# Text Processing 

### The text preprocessing steps followed in this project are:

- Tokenization
- Lower casing
- Stop words removal
- Filling Missing Data by Keyword Extraction and Entity Recognition
- Entities, URL Links and Punctuation Removal
- Spelling Correction
- Lemmatization

In [266]:
# initialise the stop words
# downloaded from https://github.com/igorbrigadir/stopwords/blob/master/en/terrier.txt 
# alternatively can use nltk.download("stopwords")
with open('helper/stopwords.txt') as f:
    stopwords = f.read().splitlines()

stop_words = set(stopwords) 

### Lemmatization with NLTK

Lemmatization is the process of grouping together the different inflected forms of a word so they can be analyzed as a single item.
Examples of lemmatization:

- rocks -> rock
- corpora -> corpus
- better -> good

In [267]:
import nltk
from builtins import str
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import emoji
from spellchecker import SpellChecker
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download("stopwords")

[nltk_data] Downloading package wordnet to /Users/sridevi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/sridevi/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /Users/sridevi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sridevi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [268]:
# initialise the Lemmatizer
lemmatizer = WordNetLemmatizer()
print("rocks :", lemmatizer.lemmatize("rocks"))
print("corpora :", lemmatizer.lemmatize("corpora"))
print("better :", lemmatizer.lemmatize("better", pos ="a"))

rocks : rock
corpora : corpus
better : good


In [309]:
# make sure some words are not flagged as misspelled
spell.word_frequency.load_words(['aftershock'])

def preprocess_text(text, ignorespelling):
    # Check if the input is a string
    if not isinstance(text, str):
        # Return non-string input as-is or convert to string
        return str(text) if text is not None else ''
    # Unquote words 
    text = urllib.parse.unquote(text)
    text = text.replace("20"," ")
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove user mentions
    text = re.sub(r"@\S+", "", text)
    # Remove hashtags
    text = re.sub(r"#\S+", "", text)
    # Remove punctuation
    text = re.sub(f"[{string.punctuation}]", "", text)
    # Remove emojis
    text = emoji.emojize(text, variant='emoji_type')
    # Lowercase the text
    text = text.lower()
    # Tokenize the text
    words = word_tokenize(text)
    # Remove stop words
    words = [w for w in words if w not in stop_words]
    # fix the spelling 
    if(ignorespelling == False):
        prevwords = words.copy()
        words[:] = [spell.correction(w) for w in words]
        # if words return a list with [None] after spell check then revert to the old list 
        for w in words:
            if w == None:
                words = prevwords   
                break;
    print(words)
    # Join the tokens back together
    return ' '.join(words)

string_with_url = "Chek out this;,, amzing:: website: https://www.example.com! #AwesomeContent @username"
string_with_no_words = "goooooooaaaaaal"

print(preprocess_text(string_with_url, False))
print(preprocess_text(string_with_no_words, False))

['check', 'amazing', 'website']
check amazing website
['goooooooaaaaaal']
goooooooaaaaaal


In [310]:
# note this code snippet takes a while to complete 

for var in categorical: 
    if var == "location":
        df[var] = df[var].apply(lambda x: preprocess_text(x, True) if pd.isna(x)!= True else x)
    else:
        df[var] = df[var].apply(lambda x: preprocess_text(x, False) if pd.isna(x)!= True else x)


['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['ablaze']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['accident']
['aftershock']
['aftershock']
['aftershock']
['aftershock']
['aftershock']
['aftershock']
['aftershock']
['aftershock']
['aftershock']
['aftershock']
['aftersho

['fatal']
['fatal']
['fatal']
['fatal']
['fatal']
['fatal']
['fatal']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatalities']
['fatality']
['fatality']
['fatality']
['fatality']
['fatality']
['fatality']
['fatality']
['fatality']
['fatality']
['fatality']
['fatality']
['fatality']
['fatality']
['fatality']
['fatality']
['fatality']
['fatality']
['fatality']
['fatality']
['fatali

['suicide', 'bomber']
['suicide', 'bomber']
['suicide', 'bomber']
['suicide', 'bomber']
['suicide', 'bomber']
['suicide', 'bomber']
['suicide', 'bomber']
['suicide', 'bomber']
['suicide', 'bomber']
['suicide', 'bomber']
['suicide', 'bomber']
['suicide', 'bomber']
['suicide', 'bomber']
['suicide', 'bomber']
['suicide', 'bomber']
['suicide', 'bomber']
['suicide', 'bombing']
['suicide', 'bombing']
['suicide', 'bombing']
['suicide', 'bombing']
['suicide', 'bombing']
['suicide', 'bombing']
['suicide', 'bombing']
['suicide', 'bombing']
['suicide', 'bombing']
['suicide', 'bombing']
['suicide', 'bombing']
['suicide', 'bombing']
['suicide', 'bombing']
['suicide', 'bombing']
['suicide', 'bombing']
['suicide', 'bombing']
['suicide', 'bombing']
['suicide', 'bombing']
['suicide', 'bombing']
['suicide', 'bombing']
['suicide', 'bombing']
['suicide', 'bombing']
['suicide', 'bombing']
['suicide', 'bombing']
['suicide', 'bombing']
['suicide', 'bombing']
['suicide', 'bombing']
['suicide', 'bombing']
['su

['india']
['mumbai']
['india']
['chicago', 'il']
['minneapolismnus']
['chicago']
['chicago', 'il']
['palo', 'alto', 'california']
['india']
['united', 'kingdom']
['nyc', 'area']
['philadelphia', 'pa']
['chicagoillinois']
['ìït', '33', '992387545328']
['lahti', 'finland']
['here']
['insane', 'asylum']
['oakland']
['macclesfield']
['michigan', 'usa']
['boulder']
['temporary', 'towers']
['macclesfield']
['pueblo']
['insane', 'asylum']
['manchester']
['stockholm', 'sweden']
['birmingham', 'uk']
['houston', 'texas']
['new', 'york']
['north', 'east', 'england']
['marvel\x89û¢dragon', 'age\x89û¢wicdiv']
['nbo']
['lagos', 'nigeria']
['st', 'louis', 'mo']
['birdland', 'new', 'meridian', 'fd']
['arizona']
['monterrey', 'mì©xico']
['infj']
['twitter']
['2b', 'hindhede', 'rd', 'singapore']
['austin', 'tx']
['kalimantan', 'timur', 'indonesia']
['prehistoric', 'earth']
['san', 'gabriel', 'la', 'union']
[]
['manila', 'philippines']
['middle', 'earth', 'asgard', 'berk']
['subconscious', 'la']
['singap

['amsterdam', 'san', 'francisco']
['australia']
['ashland', 'oregon']
['worldwide']
['riverside', 'california']
['oakland', 'ca']
['ottawa', 'canada']
['usa']
['columbus']
['world']
['washington', 'state']
['california', 'usa']
['lancaster', 'california']
['cloud', 'mafia']
['eddyville', 'oregon', '97343']
['usa']
['bakersfield', 'california']
['galveston', 'texas']
['chicago']
['palm', 'beach', 'county', 'fl']
['victoria', 'bc']
['georgia', 'tennessee']
['lagos', 'nigeria']
['puerto', 'rico']
['gettysburg', 'pa']
['austin', 'texas']
['florida', 'usa']
['hermitage', 'pa']
['jakarta', 'indonesia']
['puerto', 'rico']
['san', 'diego', 'ca']
['calgaryab']
['newton', 'nj', '07860']
['sheheryour', 'majestyempress']
['houston']
['lynbrook']
['argus', 'industries', '\\m666\\m']
['friendswood', 'tx']
['sugar', 'land', 'tx']
['property', 'universe']
['chicago']
['home', 'park']
['webster', 'tx']
['maracay', 'nirgua', 'venezuela']
['north', 'cack919']
['scottsdale', 'az']
['united', 'states']
['p

...

['yeah', 'splatoon', 'still', 'fun', 'default', 'splattershot', 'jr', 'still', 'only', 'weapon', 'layout', 'im']
['education', 'powerful', 'weapon', 'can', 'change', 'world', 'nelson']
['will', 'watch', 'dub', 'will', '100', '3', 'will', 'weapon', 'today']
['om', 'sub', 'weapon']
['weapons', 'catalogue']
['day', 'remember', 'nuclear', 'weapon', 'powerhiroshima', '70th']
['030811', 'police', '41yearold', 'bronx', 'citing', 'casing', 'victim', 'location', 'weapon', 'found']
['lol', 'look', 'tough', 'people', 'talk', 'live', 'gang', 'infested', 'cities', 'america', 'mouth', "isn't", 'weapon', 'fucking', 'wimp']
['im', 'servicing', 'section', 'im', 'lurking', 'im', 'weapon']
['iranian', 'warship', 'points', 'weapon', 'american', 'helicopter']
['agree', 'background', 'checks', 'just', 'think', 'guns', 'weapons', 'general', 'great', 'equalizer']
['hendrixonfire', 'big', 'noob', 'conquest', 'mixed', 'maps', 'weapons', 'violated', 'kdr', 'limit', 'server', 'max', '3']
['agreed', 'especially', 

['climate', 'change', 'cnn', 'weather', 'specialist', 'warningwild', 'fires', 'rain', 'flooding', 'noting', 'sun']
['thanks', 'sen', 'einstein', 'now', 'hurry', 'home', 'california', 'huge', 'uncontrolled', 'wild', 'fire', 'lightening', 'starting', 'new', 'fires']
['wild', 'fires', 'freak', 'fuck', 'like', 'hell']
['man', 'give', 'ca', 'right', 'now', 'help', 'wild', 'fires']
['europe', 'nature', 'kind', 'tornados', 'hurricanes', 'wild', 'fires', 'earthquakes', 'flash', 'floods', 'mega', 'snow', 'droughts']
['just', 'load', 'shedding', 'permanently', 'will', 'just', 'live', 'like', 'wild', 'camp', 'fires', 'cook', 'fire']
['firefighters', 'connecticut', 'headed', 'california', 'fight', 'wild', 'fires']
['wild', 'fires', 'west', 'crazy']
['deep', 'crew', 'help', 'california', 'wild', 'fires']
['control', 'wild', 'fires', 'california', 'northern', 'part', 'state', 'troubling']
['full', 'episode', 'in', '080215', 'california', 'wild', 'fires', 'force', '1', '00', 'evacuate']
['man', 'sell

['prince', 'phillip', 'numbers', 'murdered', 'british', 'amritsar', '\x89ûïthat\x89ûªs', 'bit', 'exaggerated', 'include', 'wounded\x89û\x9d']
['california', 'cops', 'sketch', 'gunman', 'who', 'killed', 'wounded', 'deputies', 'app']
['police', 'officer', 'wounded', 'suspect', 'dead', 'exchanging', 'shots', 'richmond', 'police', 'officer', 'wounded', 'suspect', 'killed', 'exchange', 'gunfire']
['lets', 'forget', 'wounded', 'female', 'veterans']
['gunmen', 'kill', 'el', 'salvador', 'bus', 'attack', 'suspected', 'salvadoran', 'gang', 'members', 'killed', 'people', 'wounded']
['seen', 'president', 'who', 'killed', 'wounded', 'child', 'man', 'crashed', 'sisters', 'plane', 'claiming', 'god']
['man', 'fatally', 'shot', 'wounded', 'vermont', 'street']
['general', 'audience', 'wounded', 'families', 'zenith', 'world', 'seen', 'rome']
['descriptions', 'wounded', 'cats', 'being', 'exciting']
['love', 'weapon', 'wounded', 'generation', 'lto']
['wounded', 'carried', 'back', 'brothers', 'sisters', 'en

['wreckage', 'conclusively', 'confirmed', 'mh370', 'malaysia', 'pm', 'investigators', 'families', 'who']
['wreckage', 'confirmed', 'part', 'missing', 'malaysia', 'airlines', 'flight', 'mh370', 'vice', 'news']
['wreckage', 'conclusively', 'confirmed', 'mh370', 'malaysia', 'pm', 'investigators', 'families', 'who']
['mh370', 'victims', 'family', 'furious', 'media', 'told', 'wreckage', 'confirmation', 'first']
['wreckage', 'conclusively', 'confirmed', 'mh370', 'malaysia', 'pm', 'investigators', 'families', 'who']
['rt', 'janeenorman', 'high', 'probability', 'aircraft', 'wreckage', 'according', 'deputy', 'prime', '\x89û']
['wreckage', 'conclusively', 'confirmed', 'mh370', 'malaysia', 'pm', 'investigators', 'families', 'who']
['wreckage', 'conclusively', 'confirmed', 'mh370', 'malaysia', 'pm', 'investigators', 'families', 'who']
['koala', 'lump', 'reuters', 'piece', 'wing', 'washed', 'indian', 'ocean', 'island', 'beach', 'last', 'week', 'part', 'wreckage', 'malaysian']
['now', 'piece', 'wrec

['m194', '0104', 'utc5km', 'volcano', 'hawaii']
['police', 'investigating', 'bike', 'collided', 'car', 'little', 'portugal', 'bike', 'rider', 'suffered', 'online', 'threatening', 'injuries']
['latest', 'homes', 'razed', 'northern', 'california', 'wildfire', 'abc', 'news']


In [311]:
#storing the results in a CSV file for futher processing since the the Text processing step takes a long time
df.to_csv('nlp-getting-started/train_with_text_processed.csv',index=False)

In [312]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,deeds reason may allah forgive,1
1,4,NaN,NaN,forest fire near la range ask canada,1
2,5,NaN,NaN,residents asked shelter place being notified officers evacuation shelter place orders expected,1
3,6,NaN,NaN,13000 people receive evacuation orders california,1
4,7,NaN,NaN,just photo ruby smoke pours school,1


In [313]:
df[categorical].isnull().sum()

keyword       61
location    2482
text           0
dtype: int64

In [314]:
df2 = pd.read_csv('nlp-getting-started/train_with_text_processed.csv')

In [315]:
df2.head()

,id,keyword,location,text,target
0,1,NaN,NaN,deeds reason may allah forgive,1
1,4,NaN,NaN,forest fire near la range ask canada,1
2,5,NaN,NaN,residents asked shelter place being notified officers evacuation shelter place orders expected,1
3,6,NaN,NaN,13000 people receive evacuation orders california,1
4,7,NaN,NaN,just photo ruby smoke pours school,1


In [316]:
df[categorical].isnull().sum()

keyword       61
location    2482
text           0
dtype: int64

### Filling missing Data 

There are missing data in both training and test dataset. 
Example in the training data following is the null value count 
- keyword       61
- location    2533
- text           0

### Keyword Extractor YAKE vs BERT

https://medium.com/@adityamishra.rishu/keyword-extractor-yake-35870de21a0d

In [333]:
pip install yake

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [339]:
import yake

text = "residents asked shelter place being notified officers evacuation shelter place orders expected"
# Specifying Parameters
language = "en"
max_ngram_size = 1
deduplication_thresold = 0.9
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 1

custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_thresold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(text)

for kw in keywords:
    print(kw)

('shelter', 0.10463417723629108)
('place', 0.10463417723629108)
('residents', 0.1161324896811994)


#### BERT keyword extractor

<center>
    <img src = images/bert_example.png width = 50%/>
</center>

https://huggingface.co/yanekyuk/bert-keyword-extractor

In [341]:
from transformers import pipeline
pipe = pipeline("token-classification", model="yanekyuk/bert-keyword-extractor")

Downloading:   0%|          | 0.00/807 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/431M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/320 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/669k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [347]:
result = pipe("residents asked shelter place being notified officers evacuation shelter place orders expected")
print(result[0])
print(result[0].get('word'))

{'entity': 'B-KEY', 'score': 0.9372667, 'index': 9, 'word': 'shelter', 'start': 65, 'end': 72}
shelter


In [379]:
# fill keyword with BERT 
for index, row in df.iterrows():
    if(pd.isnull(row['keyword'])):
        result = pipe(row['text'])
        if(len(result)>1):
            df.at[index,'keyword'] = result[0].get('word')

In [380]:
df.head()

,id,keyword,location,text,target
0,1,None,NaN,deeds reason may allah forgive,1
1,4,None,NaN,forest fire near la range ask canada,1
2,5,None,NaN,residents asked shelter place being notified officers evacuation shelter place orders expected,1
3,6,ca,NaN,13000 people receive evacuation orders california,1
4,7,rub,NaN,just photo ruby smoke pours school,1


In [381]:
#storing the results in a CSV file for futher processing since the the keyword fill step takes a long time
df.to_csv('nlp-getting-started/train_with_text_processed_keyword.csv',index=False)

### Conclusion 

The Keyword embedding would require further enhancement. Most of the keywords generated were not relevant

### What is BERT?
BERT, short for Bidirectional Encoder Representations from Transformers, is a Machine Learning (ML) model for natural language processing. It was developed in 2018 by researchers at Google AI Language and serves as a swiss army knife solution to 11+ of the most common language tasks, such as sentiment analysis and named entity recognition.

#### What is BERT used for?
BERT can be used on a wide variety of language tasks:

- Can determine how positive or negative a movie’s reviews are. (Sentiment Analysis)
- Helps chatbots answer your questions. (Question answering)
- Predicts your text when writing an email (Gmail). (Text prediction)
- Can write an article about any topic with just a few sentence inputs. (Text generation)
- Can quickly summarize long legal contracts. (Summarization)
- Can differentiate words that have multiple meanings (like ‘bank’) based on the surrounding text. (Polysemy resolution)

Reference - https://huggingface.co/blog/bert-101 

In [321]:
!pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [322]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')
unmasker("Artificial Intelligence [MASK] take over the world.")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

[{'score': 0.31823965907096863,
  'token': 2064,
  'token_str': 'can',
  'sequence': 'artificial intelligence can take over the world.'},
 {'score': 0.18299679458141327,
  'token': 2097,
  'token_str': 'will',
  'sequence': 'artificial intelligence will take over the world.'},
 {'score': 0.056001096963882446,
  'token': 2000,
  'token_str': 'to',
  'sequence': 'artificial intelligence to take over the world.'},
 {'score': 0.04519503563642502,
  'token': 2015,
  'token_str': '##s',
  'sequence': 'artificial intelligences take over the world.'},
 {'score': 0.0451534204185009,
  'token': 2052,
  'token_str': 'would',
  'sequence': 'artificial intelligence would take over the world.'}]

# BERT Embeddings

Embeddings are representations of values or objects like text, images, and audio that are designed to be consumed by machine learning models and semantic search algorithms. They translate objects like these into a mathematical form according to the factors or traits each one may or may not have, and the categories they belong to.

- https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/
- https://medium.com/mlearning-ai/getting-contextualized-word-embeddings-with-bert-20798d8b43a4

In [391]:
from transformers import BertModel, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased',
           output_hidden_states = True,)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [392]:
def bert_text_preparation(text, tokenizer):
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1]*len(indexed_tokens)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensor = torch.tensor([segments_ids])
    return tokenized_text, tokens_tensor, segments_tensor

In [393]:
def get_bert_embeddings(tokens_tensor, segments_tensor, model):
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensor)
        hidden_states = outputs[2]
    token_embeddings = torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1,0,2)
    token_vecs_sum = []
    for token in token_embeddings:
        sum_vec = torch.sum(token[-4:], dim=0)
        token_vecs_sum.append(sum_vec)
    return token_vecs_sum

In [400]:
sentence = "residents asked shelter place being notified officers evacuation shelter place orders expected"
tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(sentence, tokenizer)
embedding = get_bert_embeddings(tokens_tensor, segments_tensors, model)
print(len(embedding))
print(embedding)

14
[tensor([-8.1623e-01, -2.9886e+00,  1.0852e+00, -2.3636e-01,  6.6411e-01,
        -1.6488e-01,  9.9268e-01, -1.2012e+00,  1.4605e-01, -1.0440e+00,
         6.5752e-02, -7.0270e-01, -7.1592e-01,  1.0096e+00, -2.9296e+00,
        -7.3720e-01, -1.2393e-01,  1.8157e-01,  6.9296e-01, -3.3728e-01,
         5.0771e-01, -1.6893e+00,  5.1318e-01,  1.8093e-01,  5.9256e-01,
        -3.5719e-01, -5.7752e-01, -2.2926e-01, -6.1931e-01,  2.0697e-02,
        -1.9869e+00,  5.3706e-01, -6.6636e-01, -4.8708e-01, -3.3401e-01,
         1.5214e+00, -8.1907e-01, -1.2483e+00, -7.0475e-01,  2.2635e+00,
        -2.3427e+00, -1.0207e+00,  9.1360e-01, -4.8867e-01, -3.7359e-01,
         5.3947e-01, -1.7416e+01,  1.9159e-01, -8.0995e-01, -1.1945e+00,
         1.0682e+00, -1.4793e-01, -1.9646e-01,  4.1606e+00,  1.3556e+00,
        -7.8730e-02, -1.4198e+00, -1.2591e+00, -8.0021e-01,  1.3171e-01,
         6.3390e-01, -5.3167e-01, -9.1510e-01,  7.0456e-02,  8.0733e-01,
        -8.0783e-01, -9.5078e-01,  1.3379e+00, 